In [1]:
#Libraries required
import numpy as np
import pandas as pd
import random
import searoute as sr
from math import sin, cos, asin, sqrt, radians
import pyais
from pyais.encode import encode_dict
from datetime import datetime, timedelta
import json
import asyncio
import websockets
import logging

In [2]:
logging.basicConfig(
    filename='server.log',           # Log file name
    filemode='a',                        # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

#### Task1 - Route Generation

In [3]:
#reading the dataset
df = pd.read_csv("UpdatedPub150.csv")

In [4]:
df.head()

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,U.S. Coast Pilot 2 - Atlantic Coast: Cape Cod ...,https://nauticalcharts.noaa.gov/publications/c...,12331,...,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown,40.533333,-74.250000
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,Sailing Directions Pub. 163 (Enroute) - Borneo...,https://msi.geo.nga.mil/api/publications/downl...,,...,No,No,Unknown,No,No,NaN,Unknown,Unknown,-2.733333,121.066667
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61560,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.350000,50.000000
3,47360.0,Madagascar -- 47350,Andoany,,,Madagascar,Mozambique Channel; Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61420,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.400000,48.300000
4,47020.0,Tanzania -- 46965,Chake Chake,,,Tanzania,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61200,...,No,No,Unknown,No,No,Unknown,Unknown,Unknown,-5.250000,39.766667


In [5]:
# Cleaning the dataframe: Removing columns whose 70% values are ' ', Unknown, 0
removed_cols=[]
total_rows=len(df)

for col in df.columns:
    zero_count = (df[col]==0).sum()
    unknown_count = (df[col]=="Unknown").sum()
    empty_space_count = (df[col]==" ").sum()

    if(zero_count+unknown_count+empty_space_count) > 0.7*total_rows:
        removed_cols.append(col)

df = df.drop(columns=removed_cols)

In [6]:
df.head()

,World Port Index Number,Region Name,Main Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,Digital Nautical Chart,...,Supplies - Provisions,Supplies - Potable Water,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Deck,Supplies - Engine,Repairs,Railway,Latitude,Longitude
0,7950.0,United States E Coast -- 6585,Maurer,,United States,North Atlantic Ocean,U.S. Coast Pilot 2 - Atlantic Coast: Cape Cod ...,https://nauticalcharts.noaa.gov/publications/c...,12331,"a1707640, coa17c, h1707960",...,Yes,Yes,Yes,Yes,Yes,Yes,Moderate,Unknown,40.533333,-74.250000
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,Sailing Directions Pub. 163 (Enroute) - Borneo...,https://msi.geo.nga.mil/api/publications/downl...,,gen04c,...,No,No,No,No,No,No,NaN,Unknown,-2.733333,121.066667
2,47620.0,Madagascar -- 47350,Iharana,,Madagascar,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61560,"coa02d, gen02b",...,Yes,Yes,No,No,No,No,Emergency Only,Unknown,-13.350000,50.000000
3,47360.0,Madagascar -- 47350,Andoany,,Madagascar,Mozambique Channel; Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61420,"coa02h, gen02b",...,Unknown,Yes,No,No,No,No,Emergency Only,Unknown,-13.400000,48.300000
4,47020.0,Tanzania -- 46965,Chake Chake,,Tanzania,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61200,"coa02c, gen02a",...,No,No,No,No,No,No,Unknown,Unknown,-5.250000,39.766667


In [7]:
#To check if Latitude and Longitude columns have any NA values and subsequently removing them
df=df.dropna(subset=["Latitude","Longitude"])

In [8]:
#For n Vessels

In [9]:
n = 1# number of distinct routes to generate
routes = {}

for i in range(n):
    ports = df.sample(n=2,random_state=42+i).reset_index(drop=True)

    source = (ports.loc[0]['Longitude'], ports.loc[0]['Latitude'])
    destination = (ports.loc[1]['Longitude'], ports.loc[1]['Latitude'])

    #Printing route info
    print(f"Route{i+1}")
    print(f"Source: {ports.loc[0]['Main Port Name']} ({source[0]}, {source[1]})")
    print(f"Destination: {ports.loc[1]['Main Port Name']} ({destination[0]}, {destination[1]})\n")

    # Adding to routes dictionary
    routes[f"route{i+1}"] = {
    'source': source,
    'destination': destination,
    'source_name': ports.loc[0]['Main Port Name'],
    'destination_name': ports.loc[1]['Main Port Name']
}

Route1
Source: Pamban (79.21666667, 9.283333333)
Destination: Lockeport Harbor (-65.11666667, 43.7)



In [10]:
routes.keys()

dict_keys(['route1'])

In [11]:
routes.values()

dict_values([{'source': (79.21666667, 9.283333333), 'destination': (-65.11666667, 43.7), 'source_name': 'Pamban', 'destination_name': 'Lockeport Harbor'}])

In [12]:
#Route generation using searoute library
for i in range(n):
    route_key = f"route{i+1}"
    src = routes[route_key]['source']
    dst = routes[route_key]['destination']

    route_info = sr.searoute(src, dst, append_orig_dest=True)
    routes[route_key]['coordinates'] = route_info['geometry']['coordinates']

In [13]:
for key, data in routes.items():
    print(f"{key}")
    print(f"  From: {data['source_name']} {data['source']}")
    print(f"  To  : {data['destination_name']} {data['destination']}")
    print(f"  Total waypoints: {len(data['coordinates'])}")
    print(f"  Sample coordinates: {data['coordinates'][:2]} ...\n")


route1
  From: Pamban (79.21666667, 9.283333333)
  To  : Lockeport Harbor (-65.11666667, 43.7)
  Total waypoints: 102
  Sample coordinates: [[79.646485, 7.229672], [77, 8]] ...



#### Task2 - AIS Simulation

In [14]:
#Haversian Distance Function
#To compute distance between 2 points on earth given the co-ordiates of those points
def Haversian_distance(long1,lat1,long2,lat2):
    '''
    (long1,lat1) are the co-ordinates of the immediate previous point from the current location of the vessel
    (long2,lat2) are the co-ordinates of the immediate next point from the current location of the vessel
    '''
    long1,lat1,long2,lat2 = map(radians, [long1,lat1,long2,lat2])
    
    diff_long = long2-long1
    diff_lat = lat2-lat1

    i = sin(diff_lat/2)**2 + cos(lat1) * cos(lat2) * sin(diff_long/2)**2
    j = 2 * asin(sqrt(i))
    
    return j*6371
    # 6371 is radius of earth in km

In [15]:
#Calculating cumulative distance between all route points (helps in finding the current co-ordinates of the vessel)
def cumulative_distance(route_co_ords):
    distance=[0.0]

    for i in range(1,len(route_co_ords)):
        long1,lat1 = route_co_ords[i-1]
        long2, lat2 = route_co_ords[i]

        #Computing Haversian distace
        d = Haversian_distance(long1,lat1,long2,lat2)
        distance.append(distance[i-1]+d)
    return distance

In [16]:
#Interpolation of ship location
def interpolate_ship_location(route_co_ords, cumm_dist, curr_dist_from_source):
    position=[]
    for i in range(1,len(cumm_dist)):
        if curr_dist_from_source <= cumm_dist[i]:
            cdist0,cdist1 = cumm_dist[i-1],cumm_dist[i]
            
            dist_travelled_ratio = (curr_dist_from_source - cdist0)/ (cdist1 - cdist0)

            long0,lat0 = route_co_ords[i-1]
            long1,lat1 = route_co_ords[i]
            long = long0 + dist_travelled_ratio * (long1 - long0)
            lat = lat0 + dist_travelled_ratio * (lat1 - lat0)
            position.append(long)
            position.append(lat)

            return position
    #if curr_dist_from_source > cumm_dist[i]:
    return route_co_ords[-1]
        

In [17]:
#Simluation of ship location
def simulate_ship_position(route_co_ords, ship_speed, interval_time):
    i_time = interval_time/ 60 # Interval time in hr
    i_distance = ship_speed * i_time # distance travelled by ship in km per interval
    c_distances = cumulative_distance(route_co_ords)
    total_dist = c_distances[-1]
    num_intervals = int(total_dist// i_distance)

    positions = []
    for i in range(num_intervals+1):
        d = i * i_distance
        pos = interpolate_ship_location(route_co_ords, c_distances, d)
        positions.append(pos)

    return positions

In [18]:
# Note: In ship Speed: 1 knot is equal to 1.852 kilometers per hour (km/h)

In [19]:
#positions = simulate_ship_position(route_co_ords, 25*1.852, 5)

In [20]:
#Generating AIS Messages
def Generate_AIS_Messages(ship_positions, interval=5, ship_id=29122001):
    #ship_positions: list containing ship locations(coordinates) at regualr desired intervals.(here at every 5 min)

    start_time = datetime.utcnow() #assuming the ship left the source port at the current time(the prog runs)
    messages = []
    for i, (long, lat) in enumerate(ship_positions):
        time_stamp = (start_time + timedelta(minutes=i*5)).strftime('%d-%m-%Y %H:%M:%S') 
        msg = {
            'type' : 1,
            'mmsi': ship_id,
            'lon': long,
            'lat': lat
        }
        encoded_message = encode_dict(msg)
        messages.append((time_stamp, list(encoded_message)))

    return messages

In [21]:
vessels_messages = {}
base_mmsi = 29122001
interval_time = 5  # in minutes
ship_speed_knots = 25
ship_speed_kph = ship_speed_knots * 1.852

# Simulate each route
for i, (route_key, data) in enumerate(routes.items()):
    mmsi = base_mmsi + i
    route_co_ords = data['coordinates']

    positions = simulate_ship_position(route_co_ords, ship_speed_kph, interval_time)
    messages = Generate_AIS_Messages(positions, interval=interval_time, ship_id=mmsi)

    vessels_messages[mmsi] = messages

In [22]:
#Server
async def playback_server(vessels_messages, speed_factor=1.0, port=8765):
    
    async def handler(websocket):
        logging.info("Client connected.")
        print("Client connected.")

        try:
            # Wait for client's mmsi request
            raw_request = await websocket.recv()
            request = json.loads(raw_request)
            requested_mmsi = request.get("mmsi")

            # Check validity
            if requested_mmsi not in vessels_messages:
                error_msg = f"MMSI {requested_mmsi} not found."
                await websocket.send(json.dumps({
                    "status": "error",
                    "message": error_msg
                }))
                logging.warning(error_msg)
                return

            success_msg = f"Streaming data for MMSI {requested_mmsi}"
            await websocket.send(json.dumps({
                "status": "success",
                "message": success_msg
            }))
            logging.info(success_msg)

            send_interval = 300 / speed_factor if speed_factor > 0 else 0

            for timestamp, enc_msg in vessels_messages[requested_mmsi]:
                payload = {
                    "message": "AIVDM",
                    "mmsi": requested_mmsi,
                    "timestamp": timestamp,
                    "payload": enc_msg[0]
                }
                await websocket.send(json.dumps(payload))
                logging.info(f"Sent: {payload}")
                if speed_factor > 0:
                    await asyncio.sleep(send_interval)

        except Exception as e:
            logging.error(f"Handler error: {e}")

    async with websockets.serve(handler, "localhost", port):
        logging.info(f"Playback server started at ws://localhost:{port}")
        print(f"Playback server started at ws://localhost:{port}")
        await asyncio.Future()

In [ ]:
await playback_server(vessels_messages, speed_factor=-1, port=8000)

Playback server started at ws://localhost:8000
Client connected.
